In [2]:
#Do the basic imports for general AuxTel operations
import asyncio
from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

#And the imports needed specifically for focusing
#import numpy as np 
import matplotlib.pyplot as plt 
import time 
import numpy as np 
import math 
import scipy.optimize as optimize
%matplotlib inline 

domain = salobj.Domain()
attcs = ATTCS(domain)
latiss = LATISS(domain)
await asyncio.gather(attcs.start_task, latiss.start_task)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 97 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 97 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 20 messages
RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 20 messages
RemoteEvent(ATHeaderService, 0, heartbeat) falling behind; read 20 messages
RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 20 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 19 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 27 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 27 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 28 messages
RemoteEvent(ATDome, 0, heartbeat) falling 

[[None, None, None, None, None, None, None], [None, None, None, None]]

RemoteEvent(ATAOS, 0, detailedState) falling behind; read 36 messages
RemoteTelemetry(ATPneumatics, 0, m1AirPressure) falling behind; read 27 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 20 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 21 messages
RemoteEvent(ATPneumatics, 0, heartbeat) falling behind; read 28 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 29 messages
RemoteTelemetry(ATMCS, 0, measuredTorque) falling behind; read 29 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; re

In [2]:
await attcs.slew_object(name = "HD 90994",  rot_sky=0.0, slew_timeout=240.)

RemoteTelemetry(ATDome, 0, position) falling behind; read 35 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 89 messages


In [3]:
await latiss.take_object(exptime=5., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


array([2020021800251])

In [4]:
await attcs.offset_xy(x= -45, y= -111)

In [5]:
await latiss.take_object(exptime=5., n=10, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 24 messages


array([2020021800252, 2020021800253, 2020021800254, 2020021800255,
       2020021800256, 2020021800257, 2020021800258, 2020021800259,
       2020021800260, 2020021800261])

In [9]:
#Focus units are given in mm 
focus_center = -0.2155
focus_step = 0.02
n_steps = 9
focus_positions = np.linspace(focus_center - focus_step * math.floor(n_steps / 2), focus_center + focus_step * math.ceil(n_steps / 2 - 1), n_steps)
#focus_positions = [0.05, -0.05, -0.100, -0.150, -0.200, -0.250, -0.300, -0.3, -0.2, -0.2, -0.24, -0.26, -0.28]
print ('focus_positions = ' + str(focus_positions))

focus_positions = [-0.2955 -0.2755 -0.2555 -0.2355 -0.2155 -0.1955 -0.1755 -0.1555 -0.1355]


In [7]:
await attcs.slew_object(name = "HD 109978",  rot_sky=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 11 messages


In [8]:
await latiss.take_object(exptime=20., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


array([2020021800262])

In [9]:
await attcs.offset_xy(x= -50, y= +43)

In [ ]:
await latiss.take_object(exptime=20., n=10, filter='empty_3',grating='ronchi90lpmm')

In [10]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=5., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


Newest image id is = [2020021800263]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800264]
Working on focus position -0.2555
Newest image id is = [2020021800265]
Working on focus position -0.2355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages


Newest image id is = [2020021800266]
Working on focus position -0.2155
Newest image id is = [2020021800267]
Working on focus position -0.1955
Newest image id is = [2020021800268]
Working on focus position -0.1755


...

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021800270]
Working on focus position -0.1355


RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 11 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 39 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 12 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 12 messages


Newest image id is = [2020021800271]


RemoteEvent(ATMCS, 0, target) falling behind; read 47 messages


In [11]:
print ('science_images = ' + str(science_images))

science_images = [2020021800263, 2020021800264, 2020021800265, 2020021800266, 2020021800267, 2020021800268, 2020021800269, 2020021800270, 2020021800271]


RemoteEvent(ATMCS, 0, target) falling behind; read 80 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages


In [12]:
await attcs.slew_object(name = "HD 112618",  rot_sky=0.0, slew_timeout=240.)

RemoteEvent(ATMCS, 0, target) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages


TimeoutError: 

RemoteEvent(ATMCS, 0, target) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 41 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 20 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages


In [4]:
await attcs.slew_object(name = "HD 107899",  rot_sky=0.0, slew_timeout=240.)

In [5]:
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 34 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 51 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 36 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 31 messages


The bias data id(s) are: [2020021800272 2020021800273 2020021800274 2020021800275 2020021800276
 2020021800277 2020021800278 2020021800279 2020021800280 2020021800281]


In [6]:
await latiss.take_object(exptime=20., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


array([2020021800282])

In [7]:
await attcs.offset_xy(x= -19, y= +2)

In [10]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=5., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021800283]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800284]
Working on focus position -0.2555
Newest image id is = [2020021800285]
Working on focus position -0.2355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021800286]
Working on focus position -0.2155


...

Newest image id is = [2020021800289]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 25 messages


Newest image id is = [2020021800290]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages


Newest image id is = [2020021800291]


In [11]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=20., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955
Newest image id is = [2020021800292]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages


Newest image id is = [2020021800293]
Working on focus position -0.2555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 17 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 22 messages


Newest image id is = [2020021800294]
Working on focus position -0.2355
Newest image id is = [2020021800295]
Working on focus position -0.2155
Newest image id is = [2020021800296]
Working on focus position -0.1955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


Newest image id is = [2020021800297]
Working on focus position -0.1755
Newest image id is = [2020021800298]
Working on focus position -0.1555
Newest image id is = [2020021800299]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages


Newest image id is = [2020021800300]


In [12]:
await attcs.slew_object(name = "HD 109978",  rot_sky=0.0, slew_timeout=240.) 

In [13]:
await latiss.take_object(exptime=20., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 24 messages


array([2020021800301])

In [14]:
await attcs.offset_xy(x= -13, y= -3)

In [15]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=20., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 19 messages


Newest image id is = [2020021800302]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages


Newest image id is = [2020021800303]
Working on focus position -0.2555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages


Newest image id is = [2020021800304]
Working on focus position -0.2355
Newest image id is = [2020021800305]
Working on focus position -0.2155
Newest image id is = [2020021800306]
Working on focus position -0.1955
Newest image id is = [2020021800307]
Working on focus position -0.1755
Newest image id is = [2020021800308]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 34 messages


Newest image id is = [2020021800309]
Working on focus position -0.1355
Newest image id is = [2020021800310]


In [16]:
await attcs.slew_object(name = "HD 83832",  rot_sky=0.0, slew_timeout=240.) 

RemoteEvent(ATMCS, 0, target) falling behind; read 16 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 16 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


In [17]:
await latiss.take_object(exptime=30., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


array([2020021800311])

In [18]:
await attcs.offset_xy(x= -35, y= -29)

In [19]:
await latiss.take_object(exptime=30., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800312])

In [20]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=30., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages


Newest image id is = [2020021800313]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800314]
Working on focus position -0.2555
Newest image id is = [2020021800315]
Working on focus position -0.2355
Newest image id is = [2020021800316]
Working on focus position -0.2155


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 29 messages


Newest image id is = [2020021800317]
Working on focus position -0.1955
Newest image id is = [2020021800318]
Working on focus position -0.1755
Newest image id is = [2020021800319]
Working on focus position -0.1555


RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 20 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 32 messages


Newest image id is = [2020021800320]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages


Newest image id is = [2020021800321]


In [22]:
await attcs.slew_object(name = "HD 307630",  rot_sky=0.0, slew_timeout=240.) 

In [23]:
await latiss.take_object(exptime=30., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 22 messages


array([2020021800322])

In [24]:
await attcs.offset_xy(x= -32, y= -23)

In [25]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=30., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 20 messages


Newest image id is = [2020021800323]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 17 messages


Newest image id is = [2020021800324]
Working on focus position -0.2555
Newest image id is = [2020021800325]
Working on focus position -0.2355


...

Newest image id is = [2020021800328]
Working on focus position -0.1755
Newest image id is = [2020021800329]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 52 messages


Newest image id is = [2020021800330]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages


Newest image id is = [2020021800331]


In [26]:
await attcs.slew_object(name = "HD 112618",  rot_sky=0.0, slew_timeout=240.) 

In [27]:
await latiss.take_object(exptime=15., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 19 messages


array([2020021800332])

In [28]:
await attcs.offset_xy(x= -91, y= -59)

In [29]:
await latiss.take_object(exptime=15., n=1, filter='empty_3',grating='ronchi90lpmm')

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 17 messages


array([2020021800333])

In [30]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages


Newest image id is = [2020021800334]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800335]
Working on focus position -0.2555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


Newest image id is = [2020021800336]
Working on focus position -0.2355


...

Newest image id is = [2020021800340]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages


Newest image id is = [2020021800341]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 41 messages


Newest image id is = [2020021800342]


In [31]:
await attcs.slew_object(name = "HD 185975",  rot_sky=0.0, slew_timeout=240.) 

RemoteEvent(ATMCS, 0, target) falling behind; read 18 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 19 messages


In [32]:
await latiss.take_object(exptime=15., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800343])

In [33]:
await attcs.offset_xy(x= -22, y= 5.5)

In [34]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 28 messages


Newest image id is = [2020021800344]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800345]
Working on focus position -0.2555
Newest image id is = [2020021800346]
Working on focus position -0.2355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021800347]
Working on focus position -0.2155


...

Newest image id is = [2020021800348]
Working on focus position -0.1955
Newest image id is = [2020021800349]
Working on focus position -0.1755
Newest image id is = [2020021800350]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021800351]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 26 messages


Newest image id is = [2020021800352]


In [35]:
await attcs.slew_object(name = "HD 116351",  rot_sky=0.0, slew_timeout=240.) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 12 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 12 messages


In [36]:
await latiss.take_object(exptime=15., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800353])

In [37]:
await attcs.offset_xy(x= -77, y= 12)

In [38]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955
Newest image id is = [2020021800354]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages


Newest image id is = [2020021800355]
Working on focus position -0.2555
Newest image id is = [2020021800356]
Working on focus position -0.2355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 21 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 28 messages


Newest image id is = [2020021800357]
Working on focus position -0.2155
Newest image id is = [2020021800358]
Working on focus position -0.1955


...

Newest image id is = [2020021800359]
Working on focus position -0.1755
Newest image id is = [2020021800360]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 25 messages


Newest image id is = [2020021800361]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 28 messages


Newest image id is = [2020021800362]


In [40]:
await attcs.slew_object(name = "HD 116591",  rot_sky= -180.0, slew_timeout=240.) 

In [41]:
await latiss.take_object(exptime=15., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800363])

In [42]:
await attcs.offset_xy(x= -14, y= -53)

In [43]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=15., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 42 messages


Newest image id is = [2020021800364]
Working on focus position -0.27549999999999997


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 23 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages


Newest image id is = [2020021800365]
Working on focus position -0.2555
Newest image id is = [2020021800366]
Working on focus position -0.2355
Newest image id is = [2020021800367]
Working on focus position -0.2155
Newest image id is = [2020021800368]
Working on focus position -0.1955


...

Newest image id is = [2020021800369]
Working on focus position -0.1755
Newest image id is = [2020021800370]
Working on focus position -0.1555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 13 messages


Newest image id is = [2020021800371]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 15 messages


Newest image id is = [2020021800372]


In [44]:
await attcs.slew_object(name = "HD 109978",  rot_sky= -180.0, slew_timeout=240.) 

In [45]:
await latiss.take_object(exptime=20., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800373])

In [46]:
await attcs.offset_xy(x= 17, y= -75)

In [47]:
science_images = [0 for focus_pos in focus_positions]
for i in range(len(focus_positions)): 
    focus_pos = focus_positions[i] 
    print ('Working on focus position ' + str(focus_pos))
    attcs.athexapod.evt_positionUpdate.flush() 
    await attcs.ataos.cmd_applyFocusOffset.set_start(offset=focus_pos)
    await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=attcs.long_timeout)
    science_image = await latiss.take_engtest(exptime=20., n=1, filter='empty_3', grating='ronchi90lpmm') 
    print ('Newest image id is = ' +str(science_image) )
    science_images[i] = int(science_image)
#This should reset the focus position after taking this sequence: 
await attcs.ataos.cmd_applyFocusOffset.set_start(offset = focus_center)

Working on focus position -0.2955


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 29 messages


Newest image id is = [2020021800374]
Working on focus position -0.27549999999999997
Newest image id is = [2020021800375]
Working on focus position -0.2555


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 19 messages


Newest image id is = [2020021800376]
Working on focus position -0.2355


...

Newest image id is = [2020021800379]
Working on focus position -0.1755


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


Newest image id is = [2020021800380]
Working on focus position -0.1555
Newest image id is = [2020021800381]
Working on focus position -0.1355


RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 14 messages


Newest image id is = [2020021800382]


In [48]:
await attcs.slew_object(name = "NGC 4950",  rot_sky= 0.0, slew_timeout=240.) 

RemoteEvent(ATMCS, 0, target) falling behind; read 17 messages
RemoteEvent(ATMCS, 0, target) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 17 messages


In [49]:
await latiss.take_object(exptime=30., n=1, filter='empty_3',grating='ronchi90lpmm')

array([2020021800383])

In [50]:
bias_data_id_list = await latiss.take_bias(nbias=10)
print ('The bias data id(s) are: ' + str(bias_data_id_list)) 

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 56 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 29 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 11 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 30 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 12 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 18 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 19 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages
RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 16 messages


The bias data id(s) are: [2020021800384 2020021800385 2020021800386 2020021800387 2020021800388
 2020021800389 2020021800390 2020021800391 2020021800392 2020021800393]


In [52]:
await attcs.enable()

In [53]:
await attcs.shutdown() 

RuntimeError: Shutter Door state is 5. expected either 2 or 1